In [1]:
import pandas as pd
import numpy as np

import trecs
from trecs.models import ImplicitMF, ImplicitMFLFD
from trecs.random import Generator
from trecs.metrics import MSEMeasurement, AverageFeatureScoreRange

IndentationError: unindent does not match any outer indentation level (mflfd.py, line 103)

In [ ]:
mf = ImplicitMF(num_users=200, num_items=50, num_latent_factors=20)

In [ ]:
mflfd = ImplicitMFLFD(num_users=200, num_items=50, num_latent_factors=20)